# NYC 311 Calls: Data Preparation
The NYC Open Data site maintains a complete record of all requests made to the 311 Service beginning January 1, 2010.  The dataset includes over 36 million records with over 40 features.  The file is over 20GB.  Reading it in its entirely taxes most computers and can take a very long time (over 20 minutes on my computer).  To ease the operation of the main notebook, this notebook reads in only the data necessary for this analysis, performs all data processing necessary and converts the data into pickle files, reducing total size to under 1GB.  Reading operations should also be greatly improved (less than 6 seconds on my computer).

In [1]:
import pandas as pd
import time

In [2]:
%%time
# Read in 311 data

df_311_calls  = pd.read_csv('Data/311_Service_Requests_20240430.csv',
                            index_col='Unique Key',
                            usecols = ['Agency','Complaint Type','Created Date','Incident Zip','Unique Key'],
                            dtype = {'Agency':'category','Complaint Type':str,'Created Date':str,'Incident Zip':str,'Unique Key':'int64'})

df_311_calls = df_311_calls.rename(columns={'Complaint Type': 'Type', 'Created Date': 'Date', 'Incident Zip': 'Zip'})

CPU times: total: 43.6 s
Wall time: 1min 3s


In [3]:
%%time
# Read in NYC zip codes
df_zips = pd.read_csv('Data/uszips/uszips.csv',
                      index_col = 'zip',
                      usecols=['lat','lng','population','density','zip'],
                      dtype = {'zip': str} )

df_zips = df_zips.rename(columns={'population': 'zip_population', 'density': 'zip_density'})

CPU times: total: 0 ns
Wall time: 13.1 ms


In [4]:
print(f'Initial items: {len(df_311_calls):,.0f}')

Initial items: 36,217,243


In [5]:
%%time
# Drop rows with no useable location data
no_loc = df_311_calls['Zip'].isna() #& df_311['Latitude'].isna()
df_311_calls = df_311_calls[~no_loc]
print(f'Rows with locations: {len(df_311_calls):,.0f}')

Rows with locations: 34,682,301
CPU times: total: 984 ms
Wall time: 1.31 s


In [6]:
%%time
# Convert date columns to datetime, dropping time component and dropping dates after March 31, 2024
date_format = '%m/%d/%Y %I:%M:%S %p'
df_311_calls['Date'] = pd.to_datetime(df_311_calls['Date'], format=date_format).values.astype('datetime64[D]')
df_311_calls = df_311_calls[df_311_calls['Date'] < pd.Timestamp('2024-04-01')]
print(f'Rows in the date range: {len(df_311_calls):,.0f}')

Rows in the date range: 34,435,921
CPU times: total: 31.4 s
Wall time: 54.7 s


In [7]:
%%time
# Convert NaN values in 'Incident Zip' to 'empty', to simplify further processing
df_311_calls['Zip'] = df_311_calls['Zip'].fillna('empty')
df_311_calls.isna().sum()

# Clean up ZIP codes by removing '-####' if present
df_311_calls['Zip'] = df_311_calls['Zip'].str.replace(r'-\d{4}$', '', regex=True)

# Drop all rows with zip codes not in NYC
df_311_calls = df_311_calls[df_311_calls['Zip'].isin(df_zips.index)]
print(f'Rows in NYC: {len(df_311_calls):,.0f}')

Rows in NYC: 34,388,549
CPU times: total: 6.44 s
Wall time: 9.27 s


In [8]:
%%time
# Consolidate agencies, replacing minor agencies with 'Other'
agency_dict = {
    'NYPD': 'Police',
    'HPD': 'Housing',
    'DSNY': 'Sanitation',
    'DOT': 'Transportation',
    'DEP': 'Environment',
    'DOB': 'Buildings',
    'DPR': 'Parks',
    'DOMHM': 'Health',
    'DOF': 'Finance',
    'TLC': 'Taxi',
    'DHS': 'Homeless',
    'DCA': 'Consumer',
    'DEPARTMENT OF CONSUMER AND WORKER PROTECTION': 'Consumer',
    'HRA': 'Human Resources',
}

df_311_calls['Agency'] = df_311_calls['Agency'].map(agency_dict).fillna('Other')

CPU times: total: 859 ms
Wall time: 1.14 s


In [9]:
%%time
# Convert all line items to lowercase
df_311_calls['Type'] = df_311_calls['Type'].str.lower()

CPU times: total: 1.5 s
Wall time: 1.97 s


In [10]:
# Convert complaint types to fewer categories
complaint_dict = {
    'noise - residential': 'Noise','illegal parking': 'Vehicle','heat/hot water': 'Resident Utility','blocked driveway': 'Vehicle','street condition': 'Street Condition','noise - street/sidewalk': 'Noise',
    'street light condition': 'Traffic Device','request large bulky item collection': 'Item Pickup','plumbing': 'Resident Utility','heating': 'Resident Utility','water system': 'Resident Utility',
    'unsanitary condition': 'Sanitation','noise': 'Noise','general construction/plumbing': 'Resident Utility','traffic signal condition': 'Traffic Device','noise - commercial': 'Noise',
    'paint/plaster': 'Buildings','noise - vehicle': 'Noise','general construction': 'Construction','sewer': 'Sewer','damaged tree': 'Tree','rodent': 'Sanitation','dirty conditions': 'Sanitation',
    'electric': 'Resident Utility','derelict vehicles': 'Vehicle','sanitation condition': 'Sanitation','door/window': 'Buildings','paint - plaster': 'Buildings','sidewalk condition': 'Street Condition',
    'water leak': 'Resident Utility','building/use': 'Buildings','missed collection (all materials)': 'Sanitation','literature request': 'Other','consumer complaint': 'Consumer','general': 'Other',
    'homeless person assistance': 'Social Services','nonconst': 'Other','abandoned vehicle': 'Vehicle','new tree request': 'Tree','flooring/stairs': 'Buildings','graffiti': 'Quality of Life',
    'overgrown tree/branches': 'Tree','non-emergency police matter': 'Police','derelict vehicle': 'Vehicle','maintenance or facility': 'Other','taxi complaint': 'Taxi','appliance': 'Buildings',
    'elevator': 'Buildings','broken muni meter': 'Parking','missed collection': 'Sanitation','noise - helicopter': 'Noise','root/sewer/sidewalk condition': 'Street Condition','food establishment': 'Vendors',
    'for hire vehicle complaint': 'Taxi','dirty condition': 'Sanitation','air quality': 'Environmental','benefit card replacement': 'Social Services','encampment': 'Social Services',
    'dof property - reduction issue': 'Finance','lead': 'Public Health','safety': 'Other','street sign - damaged': 'Traffic Device','illegal fireworks': 'Quality of Life','snow': 'Snow',
    'electronics waste appointment': 'Item Pickup','scrie': 'Housing','dead/dying tree': 'Tree','illegal dumping': 'Sanitation','broken parking meter': 'Parking','other enforcement': 'Police',
    'dof parking - payment issue': 'Finance','indoor air quality': 'Buildings','noise - park': 'Noise','curb condition': 'Street Condition','street sign - missing': 'Traffic Device',
    'noncompliance with phased reopening': 'Public Health','dof property - owner issue': 'Finance','traffic': 'Traffic Device','dof property - request copy': 'Finance','highway condition': 'Street Condition',
    'dead tree': 'Tree','emergency response team (ert)': 'Buildings','special projects inspection team (spit)': 'Construction','water conservation': 'Parks and Rec','dof property - payment issue': 'Finance',
    'hpd literature request': 'Housing','animal abuse': 'Animal','housing - low income senior': 'Housing','drug activity': 'Police','vending': 'Quality of Life','electrical': 'Resident Utility',
    'illegal tree damage': 'Tree','dhs advantage - tenant': 'Housing','food poisoning': 'Public Health','street sign - dangling': 'Traffic Device','violation of park rules': 'Parks and Rec',
    'homeless encampment': 'Social Services','smoking': 'Public Health','animal-abuse': 'Animal','animal in a park': 'Animal','hazardous materials': 'Environmental','obstruction': 'Vehicle',
    'construction': 'Construction','lost property': 'Other','litter basket / request': 'Sanitation','boilers': 'Resident Utility','construction safety enforcement': 'Construction',
    'dhs advantage -landlord/broker': 'Housing','unsanitary animal pvt property': 'Animal','drinking': 'Social Services','residential disposal complaint': 'Sanitation','standing water': 'Environmental',
    'electronics waste': 'Item Pickup','panhandling': 'Social Services','dca / doh new license application request': 'Vendors','asbestos': 'Buildings','dead animal': 'Animal',
    'dof property - update account': 'Finance','outdoor dining': 'Vendors','best/site safety': 'Construction','street sweeping complaint': 'Street Condition','water quality': 'Resident Utility',
    'vendor enforcement': 'Vendors','real time enforcement': 'Buildings','housing options': 'Housing','bus stop shelter complaint': 'Social Services','industrial waste': 'Environmental',
    'bike/roller/skate chronic': 'Quality of Life','mobile food vendor': 'Vendors','overflowing litter baskets': 'Sanitation','derelict bicycle': 'Vehicle','noise survey': 'Noise','non-residential heat': 'Buildings',
    'miscellaneous categories': 'Other','homeless street condition': 'Social Services','noise - house of worship': 'Noise','taxi report': 'Taxi','traffic/illegal parking': 'Vehicle','unleashed dog': 'Animal',
    'indoor sewage': 'Resident Utility','outside building': 'Other','snow or ice': 'Snow','dof parking - request status': 'Finance','unsanitary pigeon condition': 'Animal','sweeping/missed': 'Street Condition',
    'dof parking - request copy': 'Finance','recycling enforcement': 'Sanitation','dpr internal': 'Parks and Rec','urinating in public': 'Social Services','litter basket request': 'Sanitation',
    'day care': 'Social Services','illegal posting': 'Quality of Life','borough office': 'Other','sanitation worker or vehicle complaint': 'Sanitation','vaccine mandate non-compliance': 'Public Health',
    'covid-19 non-essential construction': 'Construction','building marshals office': 'Buildings','commercial disposal complaint': 'Sanitation','dof parking - tax exemption': 'Finance',
    'bridge condition': 'Street Condition','sustainability enforcement': 'Sanitation','taxi compliment': 'Taxi','elder abuse': 'Social Services','sweeping/missed-inadequate': 'Street Condition',
    'disorderly youth': 'Social Services','abandoned bike': 'Vehicle','mold': 'Buildings','dumpster complaint': 'Sanitation','illegal animal kept as pet': 'Animal','mosquitoes': 'Environmental',
    'for hire vehicle report': 'Taxi','drie': 'Housing','found property': 'Other','home delivered meal - missed delivery': 'Social Services','litter basket complaint': 'Sanitation',
    'construction lead dust': 'Environmental','mass gathering complaint': 'Quality of Life','cranes and derricks': 'Construction','dof property - rpie issue': 'Finance','posting advertisement': 'Quality of Life',
    'home repair': 'Buildings','harboring bees/wasps': 'Animal','scaffold safety': 'Construction','adopt-a-basket': 'Sanitation','senior center complaint': 'Social Services','plant': 'Other',
    'window guard': 'Buildings','sewer maintenance': 'Sewer','snow removal': 'Snow','sweeping/inadequate': 'Street Condition',"alzheimer's care": 'Social Services','beach/pool/sauna complaint': 'Parks and Rec',
    'city vehicle placard complaint': 'Other','drinking water': 'Resident Utility','collection truck noise': 'Sanitation','question': 'Other','facades': 'Buildings','private or charter school reopening': 'Public Health',
    'dof property - property value': 'Finance','dof parking - dmv clearance': 'Finance','request xmas tree collection': 'Item Pickup','poison ivy': 'Environmental','oem disabled vehicle': 'Vehicle',
    'uprooted stump': 'Tree','wood pile remaining': 'Construction','face covering violation': 'Public Health','tattooing': 'Other','heap assistance': 'Resident Utility','highway sign - damaged': 'Traffic Device',
    'quality of life': 'Quality of Life','utility program': 'Resident Utility','executive inspections': 'Buildings','forms': 'Other','dhs advantage - third party': 'Public Health','unsanitary animal facility': 'Vendors',
    'green taxi complaint': 'Taxi','dof property - city rebate': 'Finance','forensic engineering': 'Buildings','weatherization': 'Buildings','pet shop': 'Vendors','animal facility - no permit': 'Vendors',
    'special natural area district (snad)': 'Parks and Rec','municipal parking facility': 'Parking','home delivered meal complaint': 'Social Services','illegal animal sold': 'Animal',
    'seasonal collection': 'Item Pickup','dep street condition': 'Street Condition','stalled sites': 'Other','advocate-personal exemptions': 'Finance','highway sign - missing': 'Traffic Device',
    'ahv inspection unit': 'Vendors','e-scooter': 'Quality of Life','public toilet': 'Parks and Rec','eviction': 'Housing','fatf': 'Finance','dof parking - address update': 'Parking',
    'advocate-prop refunds/credits': 'Finance','water maintenance': 'Resident Utility','highway sign - dangling': 'Traffic Device','parking card': 'Parking','taxpayer advocate inquiry': 'Finance',
    'summer camp': 'Parks and Rec','special operations': 'Quality of Life','incorrect data': 'Other','bereavement support group': 'Social Services','advocate - other': 'Social Services','lifeguard': 'Parks and Rec',
    'squeegee': 'Quality of Life','x-ray machine/equipment': 'Other','home care provider complaint': 'Social Services','case management agency complaint': 'Social Services','atf': 'Police',
    'private school vaccine mandate non-compliance': 'Public Health','overflowing recycling baskets': 'Sanitation','comments': 'Other','cooling tower': 'Buildings','recycling basket complaint': 'Sanitation',
    'calorie labeling': 'Vendors','legal services provider complaint': 'Social Services','health': 'Public Health','radioactive material': 'Environmental','dhs income savings requirement': 'Housing',
    'institution disposal complaint': 'Sanitation','water drainage': 'Environmental','green taxi report': 'Taxi','peeling paint': 'Buildings','tunnel condition': 'Street Condition',
    'building drinking water tank': 'Buildings',
}    

In [11]:
%%time
df_311_calls['Type'] = df_311_calls['Type'].map(complaint_dict).fillna('Other')

CPU times: total: 2.25 s
Wall time: 3.17 s


In [12]:
%%time
# Read in NYC weather data, and format
df_weather = pd.read_csv('Data/NYC_weather_data.csv')
df_weather.drop('Unnamed: 0', axis=1, inplace=True)
df_weather['date'] = pd.to_datetime(df_weather['date']).values.astype('datetime64[D]')
df_weather.set_index('date',inplace=True)
df_weather.drop_duplicates(inplace=True)
df_weather = df_weather[df_weather.index < pd.Timestamp('2024-04-01')]

CPU times: total: 15.6 ms
Wall time: 23.9 ms


In [13]:
%%time
df_311_calls['Agency'] = df_311_calls['Agency'].astype('category')
df_311_calls['Type'] = df_311_calls['Type'].astype('category') 

CPU times: total: 2.03 s
Wall time: 2.51 s


In [14]:
%%time
df_311_calls.to_pickle('Data/311_Calls.pickle')
df_zips.to_pickle('Data/NYC_Zips.pickle')
df_weather.to_pickle('Data/NYC_Weather.pickle')

CPU times: total: 719 ms
Wall time: 1.67 s


In [15]:
# Load shapefile of zip codes
#filtered_shapefile_path = 'Data/USA_ZIP_Code_Boundaries/filtered_zip_code_shapefile.shp'
#filtered_zip_codes_gdf = gpd.read_file(filtered_shapefile_path)

In [16]:
# Fill in latitude and longitude when a zip code is available

# Latitude and longitude from US Zip Code file
#zip_to_lat = df_zip['lat'].to_dict()
#zip_to_lng = df_zip['lng'].to_dict()

# Fill in missing latitude and longitude using the mapping, but only if values don't already exist
#df_311.loc[df_311['Latitude'].isna(), 'Latitude'] = df_311['Zip'].map(zip_to_lat)
#df_311.loc[df_311['Longitude'].isna(), 'Longitude'] = df_311['Zip'].map(zip_to_lng)